In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture --no-stderr
!pip install python-dotenv langchain-core langchain langchain-openai langchain-chroma

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

In [ ]:
# <`get_format_instructions` 메서드>

# 라이브러리 불러오기
from langchain_core.output_parsers import JsonOutputParser
# Json 출력 파서 불러오기
parser = JsonOutputParser()
instructions = parser.get_format_instructions()
print(instructions)  # JSON 형식의 지침을 출력


Return a JSON object.


In [ ]:
# <`parse` 메서드>

ai_response = '{"이름": "김철수", "나이": 30}'
parsed_response = parser.parse(ai_response)
print(parsed_response)

{'이름': '김철수', '나이': 30}


In [ ]:
# <`parse_with_prompt` 메서드>

# 라이브러리 불러오기
from langchain.output_parsers import RetryWithErrorOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
# 파서 설정
parser = RetryWithErrorOutputParser.from_llm(parser=JsonOutputParser(), llm=ChatOpenAI())

question = "가장 큰 대륙은?"
ai_response = "아시아입니다."  # JSON 형식이 아닌 잘못된 응답
try:
    result = parser.parse_with_prompt(ai_response, question)
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")
    # 여기서 AI에게 다시 질문할 수 있습니다


오류 발생: 'str' object has no attribute 'to_string'


In [ ]:
# <Pydantic 출력 파서 예시>

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, model_validator

# OpenAI 모델 설정
model = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")
    # Pydantic을 사용한 사용자 정의 검증 로직
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup", "")
        if not setup.endswith("?"):
            raise ValueError("잘못된 질문 형식입니다! 질문은 '?'로 끝나야 합니다.")
        return values

# 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해 주세요.\n{format_instructions}\n질문: {query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# 언어 모델을 사용해 데이터 구조를 채우도록 프롬프트와 모델 설정
chain = prompt | model | parser


# 체인 실행 및 결과 출력
try:
    result = chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있는 질문하여라."})
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")


setup='부동산 투자를 고려하고 있습니다. 현재 시장 상황에서 부동산에 투자하는 것이 좋은 결정일까요?' advice='부동산 투자는 장기적인 관점에서 안정적인 수익을 제공할 수 있지만, 시장의 변동성과 지역별 특성을 고려해야 합니다. 현재 시장 상황, 금리, 지역 개발 계획 등을 분석하여 투자 결정을 내리는 것이 중요합니다. 전문가와 상담하여 구체적인 전략을 세우는 것도 좋은 방법입니다.'


In [ ]:
# <SimpleJsonOutputParser 출력 파서 예시>

from langchain.output_parsers.json import SimpleJsonOutputParser
# JSON 포맷의 응답을 생성하는 프롬프트 템플릿 설정
json_prompt = PromptTemplate.from_template(
    "다음 질문에 대한 답변이 포함된 JSON 객체를 반환하십시오: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

# 스트리밍 예시: 질문에 대한 답변이 부분적으로 구문 분석됨
list(json_chain.stream({"question": "비트코인에 대한 짧은 한문장 설명."}))


[{},
 {'description': ''},
 {'description': '비'},
 {'description': '비트'},
 {'description': '비트코'},
 {'description': '비트코인은'},
 {'description': '비트코인은 분'},
 {'description': '비트코인은 분산'},
 {'description': '비트코인은 분산형'},
 {'description': '비트코인은 분산형 디'},
 {'description': '비트코인은 분산형 디지털'},
 {'description': '비트코인은 분산형 디지털 통'},
 {'description': '비트코인은 분산형 디지털 통화'},
 {'description': '비트코인은 분산형 디지털 통화로'},
 {'description': '비트코인은 분산형 디지털 통화로,'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이 개인'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이 개인 간'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이 개인 간의'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이 개인 간의 거래'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이 개인 간의 거래를'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이 개인 간의 거래를 가능'},
 {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이 개인 간의 거래를 가능하게'},
 {'description': '비트코인은 분산형 디

In [ ]:
# <JsonOutputParser출력 파서 예시>

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
# 오픈AI 모델 설정
model = ChatOpenAI(temperature=0)


# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
    setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice: str = Field(description="질문을 해결하기 위한 금융 답변")

# JSON 출력 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = JsonOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해 주세요.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 체인 구성: 프롬프트 -> 모델 -> 파서
chain = prompt | model | parser
# 체인 실행
chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있는 질문하여라."})


{'setup': '부동산 투자 시 어떤 금융 상품을 활용하는 것이 좋을까요?',
 'advice': '부동산 투자 시에는 주택담보대출, 주택청약종합저축, 부동산투자신탁 등 다양한 금융 상품을 활용할 수 있습니다. 각 상품의 장단점을 고려하여 자신의 상황에 맞는 금융 상품을 선택하는 것이 중요합니다.'}